In [ ]:
!pip install --upgrade pip 
!pip install scikit-learn --upgrade 
!pip install numpy --upgrade  
!pip install --upgrade matplotlib

In [ ]:
import pandas as pd
df = pd.read_csv('data5.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [ ]:
X1 = df.drop(["game_date_est","season","game_id","game_status_text","home_team","visitor_team","home_team_id","visitor_team_id","home_team_wins","conference","conference_visitor"], axis=1)
y1 = df["home_team_wins"]

# Split our data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.4)
win_accuracy = {}

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
# κάνουμε εκπαίδευση (fit) δηλαδή ουσιαστικά υπολογίζουμε μέση τιμή και διακύμανση για όλα τα χαρακτηριστικά και κλάσεις στο training set
model = gnb.fit(X1_train, y1_train)
# η GaussianNB έχει builtin μέθοδο υπολογισμό accuracy. Αποθηκεύουμε την τιμή της στον πίνακά μας με τα αποτελέσματα από τα άλλα classifiers
win_accuracy['gaussian naive bayes'] = gnb.score(X1_test, y1_test)
# και ξανατυπώνουμε τα sorted αποτελέσματα
print("Classification Accuracy on the NBA Games Dataset (40% test set)\n")
sorted_accuracy = [(k, win_accuracy[k]) for k in sorted(win_accuracy, key=win_accuracy.get, reverse=True)]
for k, v in sorted_accuracy:
  print(k,v)

In [ ]:
# making predictions on the testing set
y1_pred = gnb.predict(X1_test)
target_names = ['home_loss', 'home_win']

# comparing actual response values (y_test) with predicted response values (y_pred)
print("Gaussian Naive Bayes model accuracy(in %):", accuracy_score(y1_test, y1_pred))
print(f"These are the results for the model\n")
print(classification_report(y1_test, y1_pred, target_names=target_names))